# Operation Short Bull's Bit-Bot

In [50]:
import robin_stocks.robinhood as r
import pandas as pd
import ta
import time
from dotenv import load_dotenv
import os
import numpy as np
import logging
import pyotp
import itertools
import threading
import time

logging.basicConfig(
    level=logging.INFO,  
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  
    ]
)

## Load in Secrets

In [51]:
# Load the .env file
load_dotenv("/Users/rosspingatore/operation_short_bull/operation_short_bull/secrets.env")

# Access the variables
api_key = os.getenv("username")
secret_key = os.getenv("password")
alpha_num_key = os.getenv("alpha_num_key")

## Define Helper Functions

In [52]:
def is_session_active():
    """Check if the Robinhood session is still active."""
    try:
        account_info = r.profiles.load_account_profile(info=None)
        return True if account_info else False
    except Exception as e:
        logging.error(f"Session check failed: {e}")
        return False

def get_crypto_data(symbol, interval='5minute', span='day'):
    """Fetch historical price data for a specific cryptocurrency."""
    try:
        data = r.crypto.get_crypto_historicals(
            symbol,
            interval=interval,
            span=span,
            bounds='24_7'
        )
        df = pd.DataFrame(data)
        df['begins_at'] = pd.to_datetime(df['begins_at'])
        df.set_index('begins_at', inplace=True)
        df['close_price'] = df['close_price'].astype(float)
        return df
    except Exception as e:
        logging.error(f"Failed to fetch data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def apply_sma_strategy(df, ticker):
    # Shorten the moving average windows for a more responsive strategy
    df['ma_short'] = df['close_price'].rolling(window=3).mean()
    df['ma_long'] = df['close_price'].rolling(window=7).mean()
    df['signal'] = 0
    # Fixing the chained assignment using .loc
    df.loc[df.index[3:], 'signal'] = np.where(
        df['ma_short'].iloc[3:] > df['ma_long'].iloc[3:], 1, 0
    )
    df['position'] = df['signal'].diff()

    # Log the latest moving averages and signal
    latest = df.iloc[-1]
    logging.info(f"Latest Close Price: {latest['close_price']}")
    logging.info(f"Latest Short MA (3): {latest['ma_short']}")
    logging.info(f"Latest Long MA (7): {latest['ma_long']}")
    logging.info(f"Latest Signal: {latest['signal']}")
    logging.info(f"Latest Position Change: {latest['position']}")

    return df

def apply_macd_strategy(df, ticker):
    # Adjust EMA spans for short-term sensitivity
    df['ema_short'] = df['close_price'].ewm(span=6, adjust=False).mean()
    df['ema_long'] = df['close_price'].ewm(span=13, adjust=False).mean()

    # Calculate MACD line and Signal line
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=5, adjust=False).mean()

    # Generate trading signals
    df['signal'] = 0
    df['signal'] = np.where(df['macd_line'] > df['signal_line'], 1, 0)
    df['position'] = df['signal'].diff()

    # Log the latest MACD values and signal
    latest = df.iloc[-1]
    logging.info(f"Latest Close Price: {latest['close_price']}")
    logging.info(f"MACD Line: {latest['macd_line']}")
    logging.info(f"Signal Line: {latest['signal_line']}")
    logging.info(f"MACD Histogram: {latest['macd_line'] - latest['signal_line']}")
    logging.info(f"Trading Signal: {'Buy' if latest['position'] == 1 else 'Sell' if latest['position'] == -1 else 'Hold'} for {ticker}")

    return df

def apply_rsi_macd_strategy(df, ticker):
    # Calculate RSI for overbought/oversold signals
    delta = df['close_price'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))

    # Adjust MACD spans for short-term sensitivity
    df['ema_short'] = df['close_price'].ewm(span=6, adjust=False).mean()
    df['ema_long'] = df['close_price'].ewm(span=13, adjust=False).mean()
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=5, adjust=False).mean()
    # Combine RSI and MACD for trade signals
    df['signal'] = 0
    df['signal'] = np.where(
        (df['rsi'] < 30) & (df['macd_line'] > df['signal_line']), 1,  # Buy
        np.where((df['rsi'] > 70) & (df['macd_line'] < df['signal_line']), -1, 0)  # Sell
    )
    df['position'] = df['signal'].diff()

    # Log the latest indicator values and signals
    latest = df.iloc[-1]
    logging.info(f"Latest Close Price: {latest['close_price']}")
    logging.info(f"RSI: {latest['rsi']}")
    logging.info(f"MACD Line: {latest['macd_line']}")
    logging.info(f"Signal Line: {latest['signal_line']}")
    logging.info(f"Trading Signal: {'Buy' if latest['position'] == 1 else 'Sell' if latest['position'] == -1 else 'Hold'} for {ticker}")

    return df

def apply_strategy_to_tickers(tickers, strategy_function):
    """Apply a trading strategy to a specific list of cryptocurrency tickers."""
    results = {}
    for symbol in tickers:
        time.sleep(3)
        df = get_crypto_data(symbol)
        if not df.empty:
            df = strategy_function(df, symbol)
            results[symbol] = df
        else:
            logging.error(f"No data available for {symbol}. Skipping...")
    return results

def execute_trades_for_tickers(results, delay_between_tickers = 5):
    """Execute trades for a specific list of cryptocurrency tickers based on strategy signals."""
    try:
        cash_balance = float(r.profiles.load_account_profile().get('crypto_buying_power', 0))

        for symbol, df in results.items():
            latest = df.iloc[-1]
            crypto_price = latest['close_price']

            # Check if latest data is valid
            if crypto_price is None or np.isnan(crypto_price):
                logging.error(f"Invalid price data for {symbol}. Skipping trade...")
                continue

            # Get current holdings for the ticker
            holdings = r.crypto.get_crypto_positions()
            crypto_holdings = get_specific_crypto_holdings(holdings, symbol)

            # Determine action based on signal
            position_change = latest['position']

            if position_change == 1:  # Buy signal
                execute_buy_order(symbol, crypto_price, cash_balance)
            elif position_change == -1:  # Sell signal
                execute_sell_order(symbol, crypto_price, crypto_holdings)
            else:
                logging.info(f"No trade executed for {symbol} at {latest.name}.")
            


    except Exception as e:
        logging.error(f"An error occurred during trade execution: {e}")

def execute_buy_order(symbol, crypto_price, cash_balance):
    """Execute a buy order for a given symbol."""
    buy_amount_usd = cash_balance * 0.10  # 10% of available cash
    if buy_amount_usd > 0:
        buy_quantity = buy_amount_usd / crypto_price
        initial_decimal_places = 7 if buy_quantity < 1 else 2
        buy_quantity = round_quantity(buy_quantity, initial_decimal_places)
        if buy_quantity <= 0:
            logging.info(f"Calculated buy quantity is non-positive for {symbol}. Skipping buy order.")
            return
        place_order_with_retry(
            order_func=r.orders.order_buy_crypto_by_quantity,
            symbol=symbol,
            quantity=buy_quantity,
            order_type='buy',
            initial_decimal_places=initial_decimal_places
        )
    else:
        logging.info(f"Insufficient buying power for {symbol}.")

def execute_sell_order(symbol, crypto_price, crypto_holdings):
    """Execute a sell order for a given symbol."""
    if crypto_holdings > 0:
        sell_quantity = crypto_holdings * 0.10  # 10% of holdings
        initial_decimal_places = 7 if sell_quantity < 1 else 2
        sell_quantity = round_quantity(sell_quantity, initial_decimal_places)
        if sell_quantity <= 0:
            logging.info(f"Calculated sell quantity is non-positive for {symbol}. Skipping sell order.")
            return
        place_order_with_retry(
            order_func=r.orders.order_sell_crypto_by_quantity,
            symbol=symbol,
            quantity=sell_quantity,
            order_type='sell',
            initial_decimal_places=initial_decimal_places
        )
    else:
        logging.info(f"No holdings of {symbol} to sell.")

def round_quantity(quantity, decimal_places):
    """Round the quantity based on the specified decimal places."""
    return round(quantity, decimal_places)


def place_order_with_retry(order_func, symbol, quantity, order_type, initial_decimal_places):
    """Place an order with retries, adjusting decimal places if precision errors occur."""
    max_retries = 6
    retry_count = 0
    decimal_places = initial_decimal_places

    while retry_count < max_retries:
        try:
            logging.info(f"Attempting to {order_type} {quantity:.{decimal_places}f} {symbol}. Retry {retry_count + 1}/{max_retries}.")
            order_response = order_func(symbol, quantity)
            logging.debug(f"Raw API response for {order_type} order: {order_response}")

            if order_response is None:
                logging.error(f"Received None response from API for {order_type} order on {symbol}. Retrying...")
            elif 'id' in order_response:
                logging.info(f"Successfully {order_type}ed {quantity:.{decimal_places}f} {symbol}.")
                break
            else:
                # Check if the error is due to precision
                error_messages = []
                if isinstance(order_response, dict):
                    # Collect error messages from the response
                    for key, messages in order_response.items():
                        if isinstance(messages, list):
                            error_messages.extend(messages)
                        elif isinstance(messages, str):
                            error_messages.append(messages)
                else:
                    error_messages.append(str(order_response))

                if any('precision' in msg.lower() for msg in error_messages):
                    logging.warning(f"Precision too high error for {symbol} {order_type} order. Reducing decimal places.")
                    decimal_places -= 1
                    if decimal_places < 0:
                        logging.error(f"Decimal places reduced below zero for {symbol} {order_type} order. Aborting...")
                        break
                    quantity = round_quantity(quantity, decimal_places)
                    if quantity <= 0:
                        logging.error(f"Quantity rounded down to zero for {symbol} {order_type} order. Aborting...")
                        break
                    # Log the updated quantity after adjusting precision
                    logging.info(f"Adjusted quantity for {symbol} {order_type}: {quantity:.{decimal_places}f} with decimal places {decimal_places}")
                else:
                    logging.error(f"{order_type.capitalize()} order failed for {symbol}: {order_response}. Retrying...")

            retry_count += 1
            logging.info(f"Waiting 5 seconds before retying the order...")
            time.sleep(5)  # Wait before retrying

        except Exception as e:
            logging.error(f"Unhandled exception while placing {order_type} order for {symbol}: {e}")
            break
    else:
        logging.error(f"Failed to {order_type} {quantity:.{decimal_places}f} {symbol} after {max_retries} retries.")



def get_specific_crypto_holdings(positions, symbol):
    """Extract holdings for a specific cryptocurrency."""
    crypto_holdings = 0.0
    if positions:
        for position in positions:
            currency = position.get('currency', {})
            currency_code = currency.get('code') or currency.get('symbol')
            if currency_code == symbol:
                quantity = position.get('quantity_available') or position.get('quantity')
                if quantity:
                    crypto_holdings = float(quantity)
                break
    return crypto_holdings

## Call Main

In [53]:

def main():
    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(),
            logging.FileHandler("crypto_trading_bot.log")
        ]
    )
    def spinner_animation(message, duration):
        """Display a spinning icon animation with a message and countdown."""
        spinner = itertools.cycle(['|', '/', '-', '\\'])  # Spinner characters
        for remaining in range(duration, 0, -1):
            print(f"\r{message} {next(spinner)} Time left: {remaining}s", end='', flush=True)
            time.sleep(1)  # Update every second
        print("\r", end='', flush=True)  # Clear the line after the spinner

    # Authenticate with Robinhood
    def login_to_robinhood():
        totp = pyotp.TOTP(alpha_num_key).now()
        login = r.authentication.login(username=api_key, password=secret_key, mfa_code=totp)
        if login is None or 'access_token' not in login:
            logging.error("Failed to log in to Robinhood.")
            return False
        logging.info("Successfully logged in to Robinhood.")
        return True

    if not login_to_robinhood():
        return  # Exit if the initial login fails

    # List of tickers to monitor
    tickers = ["DOGE", "PEPE", "XRP", "BTC", "SOL", "WIF", "ETH", "ADA", "XLM", "SHIB"]

    while True:
        try:
            # Check if session is active
            if not is_session_active():
                logging.info("Session expired, re-authenticating...")
                if not login_to_robinhood():
                    continue  # Skip this iteration if re-authentication fails

            # Apply the chosen strategy to the specified tickers
            results = apply_strategy_to_tickers(tickers, apply_rsi_macd_strategy)

            # Execute trades based on the strategy results
            execute_trades_for_tickers(results)

        except Exception as e:
            logging.error(f"An error occurred in the main loop: {e}")
        
        finally:
            # Ensure we logout cleanly at the end of each iteration
            try:
                r.logout()
                logging.info("Logged out of Robinhood.")
            except Exception as e:
                logging.error(f"Error during logout: {e}")
        
        # Wait for 5 minutes before the next iteration
        wait_duration = 300  # 5 minutes in seconds
        spinner_thread = threading.Thread(target=spinner_animation, args=("Waiting for the next iteration...", wait_duration))
        spinner_thread.start()
        spinner_thread.join()  # Wait for the spinner thread to complete

if __name__ == "__main__":
    main()

2024-11-30 16:11:16,236 - INFO - Successfully logged in to Robinhood.
2024-11-30 16:11:19,700 - INFO - Latest Close Price: 0.428765
2024-11-30 16:11:19,701 - INFO - MACD Line: 0.0007458418107364473
2024-11-30 16:11:19,702 - INFO - Signal Line: 0.0006611122010646601
2024-11-30 16:11:19,702 - INFO - MACD Histogram: 8.47296096717872e-05
2024-11-30 16:11:19,703 - INFO - Trading Signal: Hold for DOGE
2024-11-30 16:11:22,906 - INFO - Latest Close Price: 2.0525e-05
2024-11-30 16:11:22,908 - INFO - MACD Line: 2.5233558509511663e-08
2024-11-30 16:11:22,909 - INFO - Signal Line: 2.648428486299766e-08
2024-11-30 16:11:22,910 - INFO - MACD Histogram: -1.2507263534859976e-09
2024-11-30 16:11:22,911 - INFO - Trading Signal: Sell for PEPE
2024-11-30 16:11:26,206 - INFO - Latest Close Price: 1.9105
2024-11-30 16:11:26,207 - INFO - MACD Line: 0.0019145082105789246
2024-11-30 16:11:26,208 - INFO - Signal Line: 0.0007305745787637591
2024-11-30 16:11:26,208 - INFO - MACD Histogram: 0.0011839336318151656
2

2024-11-30 16:16:53,321 - ERROR - Session check failed: load_account_profile can only be called when logged in
2024-11-30 16:16:53,323 - INFO - Session expired, re-authenticating...
2024-11-30 16:16:53,572 - INFO - Successfully logged in to Robinhood.
2024-11-30 16:16:57,041 - INFO - Latest Close Price: 0.429028
2024-11-30 16:16:57,042 - INFO - MACD Line: 0.0007457680743440664
2024-11-30 16:16:57,043 - INFO - Signal Line: 0.000689330825491129
2024-11-30 16:16:57,043 - INFO - MACD Histogram: 5.64372488529375e-05
2024-11-30 16:16:57,044 - INFO - Trading Signal: Hold for DOGE
2024-11-30 16:17:00,266 - INFO - Latest Close Price: 2.057e-05
2024-11-30 16:17:00,267 - INFO - MACD Line: 2.6801338636346086e-08
2024-11-30 16:17:00,267 - INFO - Signal Line: 2.6589969454113804e-08
2024-11-30 16:17:00,268 - INFO - MACD Histogram: 2.1136918223228247e-10
2024-11-30 16:17:00,268 - INFO - Trading Signal: Buy for PEPE
2024-11-30 16:17:03,513 - INFO - Latest Close Price: 1.8975
2024-11-30 16:17:03,514 - I

Waiting for the next iteration... / Time left: 283s

KeyboardInterrupt: 

Waiting for the next iteration... \ Time left: 281s

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-67 (spinner_animation), stopped 6245396480)>>
Traceback (most recent call last):
  File "/Users/rosspingatore/.pyenv/versions/3.11.10/lib/python3.11/threading.py", line 1002, in _bootstrap
    self._bootstrap_inner()
  File "/Users/rosspingatore/.pyenv/versions/3.11.10/lib/python3.11/threading.py", line 1047, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/Users/rosspingatore/.pyenv/versions/3.11.10/lib/python3.11/threading.py", line 1359, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/Users/rosspingatore/.pyenv/versions/3.11.10/lib/python3.11/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/Users/rosspingatore/.pyenv/versions/3.11.10/lib/python3.11/site-packages/ipykernel/iostream.py", line 267, in schedule
    self._event_pipe.send(

Waiting for the next iteration... | Time left: 280s

In [ ]:
# Logout when the script is terminated (may not be reached in an infinite loop)
r.logout()
logging.info("Logged out of Robinhood.")